# Networks

## Exploring how billboards and signage affect our digital patterns

#### CDP '24: Mapping Systems, Assignment 03

### Part 1
Using a dataset of billboards and signage, we extract the metadata of the images to geolocate them, then we extract the labeled regions, and run a pre-trained model to extract the labels for them. This gives us a data set of labels to coordinates on the map.

In [1]:
%matplotlib ipympl

import pyogrio as ogr
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import geopandas as gpd
import lonboard
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_categorical_cmap
import contextily as ctx
import pandas as pd
import seaborn as sns
import numpy as np
import folium
from folium.plugins import MarkerCluster
import h3
import os
from PIL import Image, ExifTags
from PIL import Image
from PIL.ExifTags import TAGS

In [2]:

def get_image_metadata(image_path):
    image = Image.open(image_path)
    exif = image._getexif()
    metadata = {}
    if exif:
        for tag_id, value in exif.items():
            tag = ExifTags.TAGS.get(tag_id, tag_id)
            metadata[tag] = value
        
        # Rotate image if necessary
        if "Orientation" in metadata:
            orientation = metadata["Orientation"]
            if orientation == 3:
                image = image.rotate(180, expand=True)
            elif orientation == 6:
                image = image.rotate(270, expand=True)
            elif orientation == 8:
                image = image.rotate(90, expand=True)
    
    # Convert image to numpy array
    image_array = np.array(image)
    
    return metadata, image_array

def extract_lat_long(metadata):
    gps_info = metadata.get('GPSInfo', {})
    if not gps_info:
        return None

    lat = gps_info.get(2)
    lon = gps_info.get(4)
    lat_ref = gps_info.get(1)
    lon_ref = gps_info.get(3)

    if lat and lon and lat_ref and lon_ref:
        lat = float(lat[0]) + float(lat[1]) / 60 + float(lat[2]) / 3600
        lon = float(lon[0]) + float(lon[1]) / 60 + float(lon[2]) / 3600
        
        lat *= -1 if lat_ref != 'N' else 1
        lon *= -1 if lon_ref != 'E' else 1
        
        return Point(lon, lat)
    else:
        return None

def process_images_in_directory(directory):
    images = [f for f in os.listdir(directory) if f.endswith('.jpg')]
    results = []
    for image in images:
        image_path = os.path.join(directory, image)
        metadata, image_array = get_image_metadata(image_path)
        point = extract_lat_long(metadata)
        if point:
            results.append((image, point, image_array))
    return results

# Example usage
directory = '../Data/billboards-signs-and-branding.2022-08-11T185322Z/images'
results = process_images_in_directory(directory)


In [3]:
def create_geopandas_dataframe(results):
    data = {
        'Image': [result[0] for result in results],
        'Point': [result[1] for result in results],
        'ImageArray': [result[2] for result in results]
    }
    gdf = gpd.GeoDataFrame(data, geometry='Point')
    return gdf

# Example usage
gdf = create_geopandas_dataframe(results)
print(gdf)



                                  Image                        Point  \
0   WittyWorthlessIrishdraughthorse.jpg  POINT (-118.39898 34.06248)   
1    StudiousMemorableDesertpupfish.jpg  POINT (-118.34421 34.07273)   
2                 VainWebbedIchidna.jpg  POINT (-118.34425 34.07607)   
3             MistyroseWornShibainu.jpg  POINT (-118.34885 34.07605)   
4                LikelyGaseousJunco.jpg  POINT (-118.34389 34.06954)   
..                                  ...                          ...   
92         PowerfulOrdinaryHousefly.jpg  POINT (-118.34404 34.07520)   
93        FinishedTruthfulDromedary.jpg  POINT (-118.35306 34.07043)   
94      InstructiveLightyellowLarva.jpg  POINT (-118.39920 34.06712)   
95           MeagerFilthyQueenconch.jpg  POINT (-118.35143 34.07013)   
96           DistantGaseousNematode.jpg  POINT (-118.35342 34.07051)   

                                           ImageArray  
0   [[[67, 62, 43], [68, 63, 44], [65, 60, 41], [6...  
1   [[[95, 134, 191], [

In [4]:
gdf.geometry

0     POINT (-118.39898 34.06248)
1     POINT (-118.34421 34.07273)
2     POINT (-118.34425 34.07607)
3     POINT (-118.34885 34.07605)
4     POINT (-118.34389 34.06954)
                 ...             
92    POINT (-118.34404 34.07520)
93    POINT (-118.35306 34.07043)
94    POINT (-118.39920 34.06712)
95    POINT (-118.35143 34.07013)
96    POINT (-118.35342 34.07051)
Name: Point, Length: 97, dtype: geometry

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
import matplotlib.patches as patches

debug = False

# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

def get_image_labels(image_array):
    x = np.expand_dims(image_array, axis=0)
    x = preprocess_input(x)
    
    preds = model.predict(x)
    labels = decode_predictions(preds, top=3)[0]
    return labels

def extract_region(image_array, coords):
    x_min, y_min, x_max, y_max = coords
    img_height, img_width, _ = image_array.shape
    x_min = max(0, int(x_min * img_width))
    x_max = min(img_width, int(x_max * img_width))
    y_min = max(0, int(y_min * img_height))
    y_max = min(img_height, int(y_max * img_height))
    
    # Check if the region is valid
    if x_min >= x_max or y_min >= y_max:
        return None
    
    region = image_array[y_min:y_max, x_min:x_max]

    # Display the region image (for debugging)
    if debug:
        plt.imshow(region.astype(np.uint8))
        plt.axis('off')
        plt.show()
    
    # Check if the region is not empty
    if region.size == 0:
        return None
    print(region.shape)
    if region.shape[0] < 100 or region.shape[1] < 80:
        return None
    
    return region

# Paths
annotations_base = "/Users/sebastian/Locals/CDP/src/cdp-mapping-systems/Data/billboards-signs-and-branding.2022-08-11T185322Z/annotations"

# Initialize the 'Labels' column with empty lists
gdf['Labels'] = [[] for _ in range(len(gdf))]

# Iterate through every row
for idx, row in gdf.head(10).iterrows():
    image_labels = []
    annotation_path = os.path.join(annotations_base, row['Image'].replace('.jpg', '.csv'))
    
    # Load its annotation file
    if os.path.exists(annotation_path):
        annotations = pd.read_csv(annotation_path)

        img_array = row['ImageArray']
        fig, ax = plt.subplots(1)
        ax.imshow(img_array.astype(np.uint8))
        
        # Iterate through the annotated regions
        for _, annotation in annotations.iterrows():
            coords = (annotation['x1'], annotation['y1'], annotation['x2'], annotation['y2'])
            region = extract_region(img_array, coords)
            if region is None:
                continue
            
            if debug:
                # Draw rectangle on the image
                x_min, y_min, x_max, y_max = coords
                img_height, img_width, _ = img_array.shape
                x_min = max(0, int(x_min * img_width))
                x_max = min(img_width, int(x_max * img_width))
                y_min = max(0, int(y_min * img_height))
                y_max = min(img_height, int(y_max * img_height))
                
                rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(rect)
            
            if region is not None:
                labels = get_image_labels(region)
                image_labels.extend(labels) 
        
        if debug:
            plt.axis('off')
            plt.show()
    
    gdf.at[idx, 'Labels'] = image_labels

In [ ]:
# Convert the 'Labels' column to a format that GeoJSON supports
def convert_labels_to_geojson_format(labels):
    # Discard the first element of the tuple, only include the label, and remove duplicates
    filtered_labels = ', '.join(set([label[1] for label in labels]))
    return filtered_labels

gdf_filtered = gdf.copy()
gdf_filtered['Labels'] = gdf_filtered['Labels'].apply(convert_labels_to_geojson_format)



In [ ]:
# Save the GeoDataFrame to a file
output_file = "/Users/sebastian/Locals/CDP/src/cdp-mapping-systems/Data/billboards-signs-and-branding.2022-08-11T185322Z/processed_annotations.geojson"
gdf_filtered.to_file(output_file, driver='GeoJSON')
print(f"GeoDataFrame saved to {output_file}")


In [ ]:
import folium
from folium.plugins import MarkerCluster
from PIL import Image

# Create a map centered around the average coordinates
m = folium.Map(location=[gdf_filtered.geometry.y.mean(), gdf_filtered.geometry.x.mean()], zoom_start=14, tiles="Cartodb Positron")

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

base = "/Users/sebastian/Locals/CDP/src/cdp-mapping-systems/Data/billboards-signs-and-branding.2022-08-11T185322Z/images"
# Add points to the map with hover labels
for idx, row in gdf_filtered.iterrows():
    if row['Labels'] == '':
        continue
    image_path = os.path.join(base, row['Image'])
    small_image_path = os.path.join(base, f"small_{row['Image']}")
    
    # Create a small 100x100 copy of the image
    with Image.open(image_path) as img:
        img.thumbnail((200, 200))
        img.save(small_image_path)
    
    folium.Marker(
        location=[row['Point'].y, row['Point'].x],
        popup=row['Labels'],
        icon=folium.features.CustomIcon(icon_image=small_image_path, icon_size=(100,100))
    ).add_to(marker_cluster)

# Save the map to an HTML file
output_map_file = "/Users/sebastian/Locals/CDP/src/cdp-mapping-systems/Data/billboards-signs-and-branding.2022-08-11T185322Z/interactive_map.html"
m.save(output_map_file)
print(f"Interactive map saved to {output_map_file}")

# Display the map in the notebook
m


In [ ]:
import googlemaps
import os
import json

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='YOUR_API_KEY')

# Function to get the nearest place for a given location and label
def get_nearest_place(lat, lng, label):
    places_result = gmaps.places_nearby(location=(lat, lng), keyword=label, rank_by='distance')
    if places_result['results']:
        return places_result['results'][0]['name']
    return None

# File to save the results
output_file = "/Users/sebastian/Locals/CDP/src/cdp-mapping-systems/Data/billboards-signs-and-branding.2022-08-11T185322Z/nearest_places.json"

# Check if the file already exists
if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        nearest_places = json.load(f)
else:
    nearest_places = {}

# Iterate over each row and get the nearest place for the first label
for idx, row in gdf_filtered.iterrows():
    if row['Labels'] == '':
        continue
    first_label = row['Labels'].split(',')[0]
    point_key = f"{row['Point'].y},{row['Point'].x}"
    if point_key not in nearest_places:
        nearest_place = get_nearest_place(row['Point'].y, row['Point'].x, first_label)
        nearest_places[point_key] = nearest_place
        print(f"Nearest place to {first_label} at ({row['Point'].y}, {row['Point'].x}): {nearest_place}")

# Save the results to a file
with open(output_file, 'w') as f:
    json.dump(nearest_places, f)
print(f"Nearest places saved to {output_file}")
